In [2]:
%pip install DBcm --upgrade


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import DBcm

In [4]:
db_details = "CoachDB.sqlite3"

In [5]:
SQL_INSERT = """
    insert into swimmers
    (name, age)
    values
    (?, ?)
"""

In [6]:
with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL_INSERT, ("Hannal", 13))

In [10]:
with DBcm.UseDatabase(db_details) as db:
    db.execute("select * from swimmers")
    results = db.fetchall()
results

[]

In [9]:
with DBcm.UseDatabase(db_details) as db:
    db.execute("delete from swimmers")

In [11]:
import os

FOLDER = "swimdata/"

files = os.listdir(FOLDER)
files.remove(".DS_Store")

with DBcm.UseDatabase(db_details) as db:
    for fn in files:
        name, age, *_ = fn.removesuffix(".txt").split("-")
        db.execute(SQL_INSERT, (name, age))

In [13]:
SQL_SELECT = """
    select * from swimmers
    where name = ? and age = ?
"""

with DBcm.UseDatabase(db_details) as db:
    for fn in files:
        name, age, *_ = fn.removesuffix(".txt").split("-")
        db.execute(SQL_SELECT, (name, age))
        if db.fetchall():
            continue
        db.execute(SQL_INSERT, (name, age))

In [36]:
import os

import DBcm

db_details = "CoachDB.sqlite3"

FOLDER = "swimdata/"

files = os.listdir(FOLDER)
files.remove(".DS_Store")

SQL_SELECT = """
    select * from events
    where distance = ? and stroke = ?
"""

SQL_INSERT = """
    insert into events
    (distance, stroke)
    values
    (?, ?)
"""

with DBcm.UseDatabase(db_details) as db:
    for fn in files:
        _, _, distance, stroke = fn.removesuffix(".txt").split("-")
        db.execute(SQL_SELECT, (distance, stroke,))
        if db.fetchall():
            continue
        db.execute(SQL_INSERT, (distance, stroke,))

In [38]:
SQL = "select * from events"
with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL)
    results = db.fetchall()
results

[(2, '100m', 'Free'),
 (3, '100m', 'Back'),
 (4, '100m', 'Fly'),
 (5, '50m', 'Back'),
 (6, '200m', 'Free'),
 (7, '200m', 'Back'),
 (8, '50m', 'Free'),
 (9, '50m', 'Breast'),
 (10, '200m', 'IM'),
 (11, '100m', 'Breast'),
 (12, '400m', 'Free'),
 (13, '50m', 'Fly'),
 (14, '200m', 'Breast')]

In [39]:
with DBcm.UseDatabase(db_details) as db:
    db.execute("pragma table_info(times)")
    results = db.fetchall()
results

[(0, 'swimmer_id', 'INTEGER', 1, None, 0),
 (1, 'event_id', 'INTEGER', 1, None, 0),
 (2, 'time', 'varchar(16)', 1, None, 0),
 (3, 'ts', 'timestamp', 0, 'current_timestamp', 0)]

In [40]:
SQL_GET_SWIMMER = """
    select id from swimmers
    where name = ? and age = ?
"""

In [41]:
SQL_GET_EVENT = """
    select id from events
    where distance = ? and stroke = ?
"""

In [42]:
SQL_INSERT = """
    insert into times
    (swimmer_id, event_id, time)
    values
    (?, ?, ?)
"""

In [43]:
with DBcm.UseDatabase(db_details) as db:
    for fn in files:
        name, age, distance, stroke = fn.removesuffix(".txt").split("-")
        db.execute(SQL_GET_SWIMMER, (name, age))
        swimmer_id = db.fetchone()[0]
        db.execute(SQL_GET_EVENT, (distance, stroke))
        event_id = db.fetchone()[0]
        with open(FOLDER + fn) as sf:
            times = sf.read().strip().split(",")
            for t in times:
                db.execute(SQL_INSERT, (swimmer_id, event_id, t))

In [44]:
with DBcm.UseDatabase(db_details) as db:
    db.execute("select * from times")
    results = db.fetchall()
results

[(2, 2, '1:21.43', '2024-01-20 12:53:21'),
 (2, 2, '1:21.40', '2024-01-20 12:53:21'),
 (2, 2, '1:21.62', '2024-01-20 12:53:21'),
 (2, 2, '1:25.38', '2024-01-20 12:53:21'),
 (3, 3, '1:22.57', '2024-01-20 12:53:21'),
 (3, 3, '1:29.64', '2024-01-20 12:53:21'),
 (3, 3, '1:20.39', '2024-01-20 12:53:21'),
 (3, 3, '1:23.83', '2024-01-20 12:53:21'),
 (4, 2, '1:15.57', '2024-01-20 12:53:21'),
 (4, 2, '1:14.40', '2024-01-20 12:53:21'),
 (4, 2, '1:19.82', '2024-01-20 12:53:21'),
 (4, 2, '1:12.90', '2024-01-20 12:53:21'),
 (5, 2, '1:02.52', '2024-01-20 12:53:21'),
 (5, 2, '1:02.56', '2024-01-20 12:53:21'),
 (5, 2, '1:06.41', '2024-01-20 12:53:21'),
 (5, 2, '1:07.08', '2024-01-20 12:53:21'),
 (5, 2, '1:02.56', '2024-01-20 12:53:21'),
 (5, 2, '1:03.23', '2024-01-20 12:53:21'),
 (5, 2, '1:04.54', '2024-01-20 12:53:21'),
 (2, 3, '1:35.75', '2024-01-20 12:53:21'),
 (2, 3, '1:32.78', '2024-01-20 12:53:21'),
 (2, 3, '1:34.01', '2024-01-20 12:53:21'),
 (2, 3, '1:32.57', '2024-01-20 12:53:21'),
 (5, 3, '1: